In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time

In [1]:
# Creating a driver instance of selenium
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()

In [3]:
# Wait time for elements to appear under expected conditions
timeout = 5

# Locations of interest for data scraping
locations = ["Banglore, Karnataka, India",
            "Hyderabad, Telangana, India", 
             "Delhi, India", 
             "Mumbai, Maharashtra, India", 
             "Kolkata, West Bengal, India"
            ]

# Dishes of interest for data scraping
dishes = ['Chicken Biryani', 'Masala Dosa', 'Hyderabadi Haleem',
          'Bisi bele bath', 'Medu Vada', 'Curd Rice', 'Idly',
          'Mutton Rogan Josh', 'Upma', 'Pulihora', 'South Indian Meals', 'Mughlai Biryani',
          'Pepper Chicken', 'Poori', 'Butter Dosa','Paneer Dosa', 'Sambar Rice', 'Veg Biryani', 
          'Paneer Biryani', 'Uttapam','Vegetarian Pizza','Vegetarian Lasagne',
          'Chicken Pasta Salad', 'Baked Pasta','Chicken Steak',
          'Chicken Lasagne','Garlic Bread','Chicken Shawarma','Falafal & Hummus','Nachos',
          'Rissoto','Veg Sandwich', 'Chicken Sandwich','Spaghetti','Chicken Pizza',
          'Chicken Burger','Paneer Burger','Veg Burger',
          'White Sauce Pasta', 'Mac & Cheese','Chilli Chicken','Chicken Manchurian',
          'Veg Manchurian','Vegetable Chowmein', 'Manchow Soup','Spring Rolls',
          'Schezuan Chicken','Veg Momos','Chicken Momos','Fried Rice','Hakka Noodles',
          'Dimsums', 'Sweet Corn Soup', 'Chilli Paneer','Sweet And Sour Chicken',
          'Honey Chilli Potatoes','Garlic Prawns',
          'Hot and Sour Soup','Garlic Chicken','Chicken Lollipop','Aloo Gobi','Aloo Matar', 
          'Aloo Shimla Mirch','Butter Chicken','Chana Masala','Chapati','Chicken Tikka',
          'Chicken Tikka masala','Chole Bhature','Daal Puri',
          'Dal Tadka','Dum Aloo','Gobhi Matar','Kadai Paneer','Litti Chokha',
          'Makki Di Roti Sarson Da Saag', 'Mixed Vegetable','Garlic Naan','Palak Paneer',
          'Paneer Butter Masala','Paneer Tikka Masala','Rajma Chaval',
          'Shahi Paneer','Tandoori Chicken','Tandoori Fish Tikka','Gajar Ka Halwa',
          'Gulab Jamun','Kaju Katli', 'Kalakand','Kadu Ki Kheer','Phirni',
          'Double Ka Meetha','Qubani Ka Meetha','Rasgulla','Lassi','Butter Milk',
          'Strawberry Lemonade','Ice Tea','Mojito','Masala Chai','Irish Coffee',
          'Shahi Tukda','Chocolate Lava Cake', 'Vanilla Ice Cream','Milk Shake'
         ]


# loading the swiggy page
# driver.get("http://www.swiggy.com")

# Creating an empty data frame to store the final results
# restaurants_df = pd.DataFrame()

In [4]:
def shoot_crawlers(restaurants_df):
    
    for location in locations:

        # Initializing variables for the fields that are to be scraped from swiggy
        restaurants = []
        cuisine = []
        rating = []
        delivery_time = []
        cost_for_two = []
        URL = []

        # loading the swiggy page
        driver.get("http://www.swiggy.com")

        # Setting the location - pass location in the location input field
        loc_input = driver.find_element_by_id("location")
        time.sleep(2)
        loc_input.send_keys(location)
        time.sleep(2)
        driver.find_element_by_xpath("//div[text()='Clear']/following-sibling::div//div[1]").click()
        time.sleep(4)

        for dish in dishes:

            # Navigating to the search page, entering search string and choosing dishes tab
            driver.get("https://www.swiggy.com/search")

            # Passing the dish name and pressing enter
            try:
                element_present = EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search for restaurants or dishes']"))
                WebDriverWait(driver, timeout).until(element_present)
                driver.find_element_by_xpath("//input[@placeholder='Search for restaurants or dishes']").send_keys(dish, Keys.ENTER)
            except TimeoutException:
                print("Timed out waiting for page to load")

            # Moving to the dishes tab
            try:
                element_present = EC.presence_of_element_located((By.XPATH, "//button[text()='Dishes']"))
                WebDriverWait(driver, timeout).until(element_present)
                driver.find_element_by_xpath("//button[text()='Dishes']").click()
                time.sleep(2)
            except TimeoutException:
                print("Timed out waiting for page to load")

            # Fetching restaurant web elements and extracting restaurant name
            rest_elems = driver.find_elements_by_xpath('//div[@role="presentation"]//h2')
            for elem in rest_elems:
                restaurants.append(elem.text.split('\n')[0])

            # Fetching restaurant cuisine
            cuis_elems = driver.find_elements_by_xpath('//div[@role="presentation"]//p[position()=1]')
            for elem in cuis_elems:
                cuisine.append(elem.text)

            # Fetching restaurant rating
            rating_elems = driver.find_elements_by_xpath('//div[@role="presentation"]//p[position()=2]//span[position()=1]')
            for elem in rating_elems:
                rating.append(float(elem.text if elem.text != '--' else '-1'))

            # Fetching estimated delivery time
            deltime_elems = driver.find_elements_by_xpath('//div[@role="presentation"]//p[position()=2]//span[position()=2]')
            for elem in deltime_elems:
                delivery_time.append(elem.text)

            # Fetching estimated cost for two
            cost4two_elems = driver.find_elements_by_xpath('//div[@role="presentation"]//p[position()=2]//span[position()=3]')
            for elem in cost4two_elems:
                cost_for_two.append(elem.text.split(' ')[0])

            # Getting the restaurant order page URL
            rest_URLs = driver.find_elements_by_xpath('//div[@role="presentation"]//a')
            for elem in rest_URLs:
                URL.append(elem.get_attribute('href'))

            # Repeating location N times 
            loc4df = [location] * len(rest_elems)

            # Repeating dish N times
            dish4df = [dish] * len(rest_elems)

            temp_df = pd.DataFrame(list(zip(loc4df, dish4df, restaurants, cuisine, rating, 
                                    delivery_time, cost_for_two, URL)),
                                  columns = ['Location', 'Dish', 'Restaurant Name', 
                                             'Cuisine', 'Rating', 'Delivery_Time', 
                                             'Cost_for_two', 'Order_URL'])

            restaurants_df = restaurants_df.append(temp_df, ignore_index=True)

        # Before changing the location clearing out the cookies 
        driver.delete_all_cookies()
    return restaurants_df

In [5]:
restaurants_df = pd.DataFrame()
data = shoot_crawlers(restaurants_df)

In [6]:
data

,Location,Dish,Restaurant Name,Cuisine,Rating,Delivery_Time,Cost_for_two,Order_URL
0,"Banglore, Karnataka, India",Chicken Biryani,Nagarjuna,"Andhra, South Indian",4.4,26 MINS,₹600,https://www.swiggy.com/restaurants/nagarjuna-r...
1,"Banglore, Karnataka, India",Chicken Biryani,New Taj Darbar,North Indian,3.8,29 MINS,₹300,https://www.swiggy.com/restaurants/new-taj-dar...
2,"Banglore, Karnataka, India",Chicken Biryani,Bheemas,Andhra,4.4,33 MINS,₹550,https://www.swiggy.com/restaurants/bheemas-chu...
3,"Banglore, Karnataka, India",Chicken Biryani,Hamza hotel,"Continental, Fast Food, Street Food",4.2,26 MINS,₹300,https://www.swiggy.com/restaurants/hamza-hotel...
4,"Banglore, Karnataka, India",Chicken Biryani,Hyderabad Biriyani House,"Biryani, Chinese, Hyderabadi, Andhra, Fast Food",4.3,34 MINS,₹500,https://www.swiggy.com/restaurants/hyderabad-b...
...,...,...,...,...,...,...,...,...
28481,"Kolkata, West Bengal, India",Milk Shake,Kathi On The Go,"Biryani, Indian, Kebabs, Mughlai, Fast Food",3.8,69 MINS,₹300,https://www.swiggy.com/restaurants/kathi-on-th...
28482,"Kolkata, West Bengal, India",Milk Shake,Apu's Kitchen Phool Bagan,"Indian, Chinese",4.3,50 MINS,₹300,https://www.swiggy.com/restaurants/apus-kitche...
28483,"Kolkata, West Bengal, India",Milk Shake,Sabita Restaurant,"North Indian, Chinese, Bengali, Biryani, India...",4.1,72 MINS,₹300,https://www.swiggy.com/restaurants/sabita-rest...
28484,"Kolkata, West Bengal, India",Milk Shake,Royal India Restaurant,"Biryani, Chinese, Tandoor , indian, , North In...",4.2,69 MINS,₹300,https://www.swiggy.com/restaurants/royal-india...


In [7]:
data.to_csv('swiggy.csv', index = False)